In [ ]:
# Imports
import time
import numpy as np
import pandas as pd
from scipy.stats import skew, kurtosis
from scipy.signal import butter, filtfilt
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, confusion_matrix
import matplotlib.pyplot as plt
from scipy.signal import butter, filtfilt, welch
import glob

In [35]:
# For handling raw data

def load(data):
    pass

In [36]:
# Main function
def main():
    # 1. Grab the labels.txt file telling you what each data section means (x)
    # 2. For each section: (x)
        # i. Load that section's respective data
        # ii. For each window in that section:
            # a. Apply noise filters on the window
            # b. Compute features from the window
            # c. Store those feature values in an array
        # iii. Store all the features and their respective data in a file
    labels_file = LabelsFile()
    for section in labels_file:
        raw_data = load(section)
        pass
    pass

if __name__ == "__main__":
    main()